In [0]:
#Pyspark-DecisionTreeRegressor
#Import the required libraries
from __future__ import print_function
import pyspark
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler,VectorAssembler,Imputer,VectorIndexer
from pyspark.sql import SparkSession

In [0]:
#Load the data in to the local machine
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/praneethsanthosh555@gmail.com/Admission_Prediction-1.csv",header=True)

In [0]:
df1.show(5)

+---------+-----------+-----------------+----+----+----+--------+---------------+
GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
 337.00| 118.00| 4|4.50|4.50|9.65| 1.00| 0.92|
 324.00| 107.00| 4|4.00|4.50|8.87| 1.00| 0.76|
 null| 104.00| 3|3.00|3.50|8.00| 1.00| 0.72|
 322.00| 110.00| 3|3.50|2.50|8.67| 1.00| 0.80|
 314.00| 103.00| 2|2.00|3.00|8.21| 0.00| 0.65|
+---------+-----------+-----------------+----+----+----+--------+---------------+
only showing top 5 rows

In [0]:
#Get the column names
df1.columns

Out[40]: ['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [0]:
#get the data type of the variables 
df1.dtypes.

File "<command-3132988614399373>" , line 2 
 df1.dtypes. 
 ^
 SyntaxError : invalid syntax

In [0]:
from pyspark.sql.functions import col,when,count
new_df=df1.select(*(col(c).cast('float').alias(c) for c in df1.columns))
new_df.show(5)

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 337.0| 118.0| 4.0|4.5|4.5|9.65| 1.0| 0.92|
 324.0| 107.0| 4.0|4.0|4.5|8.87| 1.0| 0.76|
 null| 104.0| 3.0|3.0|3.5| 8.0| 1.0| 0.72|
 322.0| 110.0| 3.0|3.5|2.5|8.67| 1.0| 0.8|
 314.0| 103.0| 2.0|2.0|3.0|8.21| 0.0| 0.65|
+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 5 rows

In [0]:
#Check any null value are present in the data
missing_data=new_df.select([count(when(col(c).isNull(),c)).alias(c) for c in new_df.columns])
missing_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 15| 10| 15| 0| 0| 0| 0| 0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

In [0]:
#impute the missing values in the data
impute_data=Imputer(inputCols=["GRE Score","TOEFL Score","University Rating"],
                   outputCols=["GRE Score","TOEFL Score","University Rating"])
model_impute=impute_data.fit(new_df)

imputed_data=model_impute.transform(new_df)

In [0]:
#After imputeing the null values agian check is there any missing values in the data
imputed_data.select([count(when(col(c).isNull(),c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

In [0]:
features=imputed_data.drop("Chance of Admit")
#Assbmle all the fetaures in the data
assemble_data=VectorAssembler(inputCols=features.columns,outputCol="features")
assembler=assemble_data.transform(imputed_data)

In [0]:
assembler.select("features","Chance of Admit").show(5)

+--------------------+---------------+
 features|Chance of Admit|
+--------------------+---------------+
[337.0,118.0,4.0,...| 0.92|
[324.0,107.0,4.0,...| 0.76|
[316.558776855468...| 0.72|
[322.0,110.0,3.0,...| 0.8|
[314.0,103.0,2.0,...| 0.65|
+--------------------+---------------+
only showing top 5 rows

In [0]:
indexer=VectorIndexer(inputCol="features",outputCol="IndexedFeatures",maxCategories=4).fit(assembler)
indexer_df=indexer.transform(assembler)

In [0]:
new_output=indexer_df.select("IndexedFeatures","Chance of Admit")
new_output.show(5,truncate=False)

+-----------------------------------------------+---------------+
IndexedFeatures |Chance of Admit|
+-----------------------------------------------+---------------+
[337.0,118.0,4.0,4.5,4.5,9.649999618530273,1.0]|0.92 |
[324.0,107.0,4.0,4.0,4.5,8.869999885559082,1.0]|0.76 |
[316.55877685546875,104.0,3.0,3.0,3.5,8.0,1.0] |0.72 |
[322.0,110.0,3.0,3.5,2.5,8.670000076293945,1.0]|0.8 |
[314.0,103.0,2.0,2.0,3.0,8.210000038146973,0.0]|0.65 |
+-----------------------------------------------+---------------+
only showing top 5 rows

In [0]:
#Split the data for trianing and testing set
train_df,test_df=new_output.randomSplit([0.7,0.3])
#Show the train and test data
train_df.show(5)
test_df.show(5)

+--------------------+---------------+
 IndexedFeatures|Chance of Admit|
+--------------------+---------------+
[290.0,100.0,1.0,...| 0.47|
[290.0,104.0,4.0,...| 0.45|
[294.0,93.0,1.0,1...| 0.46|
[294.0,95.0,1.0,1...| 0.49|
[295.0,93.0,1.0,2...| 0.46|
+--------------------+---------------+
only showing top 5 rows

+--------------------+---------------+
 IndexedFeatures|Chance of Admit|
+--------------------+---------------+
[293.0,97.0,2.0,2...| 0.64|
[296.0,99.0,2.0,2...| 0.61|
[297.0,96.0,2.0,2...| 0.43|
[297.0,100.0,1.0,...| 0.52|
[298.0,92.0,1.0,2...| 0.51|
+--------------------+---------------+
only showing top 5 rows

In [0]:
#Create a model and fit for training
dt=DecisionTreeRegressor(featuresCol="IndexedFeatures",labelCol="Chance of Admit")
model=dt.fit(train_df)

In [0]:
#Get the prediction on the test data
predictions=model.transform(test_df)
predictions.select("prediction","Chance of Admit").show(5)

+-------------------+---------------+
 prediction|Chance of Admit|
+-------------------+---------------+
0.49315789341926575| 0.64|
 0.6225000023841858| 0.61|
0.49315789341926575| 0.43|
 0.6225000023841858| 0.52|
0.49315789341926575| 0.51|
+-------------------+---------------+
only showing top 5 rows

In [0]:
#Get the metrics on the test data
evaluter=RegressionEvaluator(labelCol="Chance of Admit",predictionCol="prediction",metricName="mae")
predictions_text=evaluter.evaluate(predictions)
print('MAE Score: %s' % predictions_text)

MAE Score: 0.05382070672597952

In [0]:
evaluter=RegressionEvaluator(labelCol="Chance of Admit",predictionCol="prediction",metricName="r2")
predictions_text=evaluter.evaluate(predictions)
print('R2 Score: %s' % predictions_text)

R2 Score: 0.7068326200161914

In [0]:
evaluter=RegressionEvaluator(labelCol="Chance of Admit",predictionCol="prediction",metricName="rmse")
predictions_text=evaluter.evaluate(predictions)
print('RMSE Score: %s' % predictions_text)

RMSE Score: 0.07678493735240635